In [26]:
using Random
using Distributions
using DataFrames
using CSV

In [3]:
function build_lattice_df(Lx,Ly)
    
    N = Lx*Ly
    sites = collect(1:N)
    dirs = ['x','y']
    interactions = DataFrame(["$site,$dir" => [] for site in sites for dir in dirs])    
    for site in 1:N
        up = mod(site-1-Lx,N)
        down = mod(site-1+Lx,N)
        right = Int(((site-1)-((site-1)%Lx)) + mod(((site-1)%Lx)+1,Lx))
        left = Int(((site-1)-((site-1)%Lx)) + mod(((site-1)%Lx)-1,Lx))
        push!(interactions[!,"$site,x"],[left+1,right+1])
        push!(interactions[!,"$site,y"],[up+1,down+1])
    end
    
    return interactions
end

build_lattice_df (generic function with 1 method)

In [4]:
function rows_and_columns(Lx,Ly)
    
    N =Lx*Ly
    rows = DataFrame(["row$row" => [] for row in 1:Ly])    
    cols = DataFrame(["col$col" => [] for col in 1:Lx])
    for site in 1:N
        row = Int((((site-1) - ((site-1)%Lx))/Ly)+1)
        col = Int(((site-1) % Lx)+1)
        push!(rows[!,"row$row"],site)
        push!(cols[!,"col$col"],site)
    end
    
    return rows, cols
end

rows_and_columns (generic function with 1 method)

In [5]:
function Calculate_XY_Energy(config,interactions)
    
    N = length(config)
    totalenergy = 0
    
    for site in (1:N)
        neighbors_x = interactions[!,"$site,x"][1,1]
        neighbors_y = interactions[!,"$site,y"][1,1]
        energy_contributions_x = 0.5*-1*cos.(config[site].-config[neighbors_x])
        energy_contributions_y = 0.5*-1*cos.(config[site].-config[neighbors_y])
        totalenergy += sum(energy_contributions_x) + sum(energy_contributions_y)
    end
    
    return totalenergy

end

Calculate_XY_Energy (generic function with 1 method)

In [6]:
function XY_update(config,energy,interactions,beta,energy_func)
    
    random_angles =rand(Float64,length(config))*2*pi
#     new_config = mod2pi.(copy(config).+random_angles)
    new_energy = energy_func(random_angles,interactions)
    deltaE = new_energy-energy
    exp_beta_deltaE = exp(-1*beta*deltaE)
    rand_value = rand(Float64,1)[1]
    delta_accept_count = 0
    
    if exp_beta_deltaE > rand_value
        config = random_angles
        delta_accept_count += 1
    else
        deltaE = 0.
    end
    
    return config, deltaE, delta_accept_count
end

XY_update_function (generic function with 1 method)

In [ ]:
function Wolff_step(L::Int,neighbours, spins_angles,T)
    # angle defining flipping axis
    N = L^2
    α = rand(Uniform(0,2*pi))
    cluster = []
    spins_to_check = []
    initial_spin = rand(1:N)
    append!(cluster, initial_spin)
    append!(spins_to_check, initial_spin)
    while !isempty(spins_to_check)
        # check all neighbours of spins_to_check
        spin_being_checked = spins_to_check[1]
        θ = spins_angles[spin_being_checked]
        all_neighbours = neighbours[spin_being_checked ,:]
        neigh_not_in_cluster = setdiff(all_neighbours,cluster)
        for i in neigh_not_in_cluster
            θ_neigh = spins_angles[i]
            prob = 1-exp(min(0,-(2/T)*cos(θ-α)*cos(θ_neigh-α)))
            # println((2/T)*cos(θ-α)*cos(θ_neigh-α))
            # println(prob)
            if rand() < prob
                append!(cluster, i)
                append!(spins_to_check, i)
            end
        end
        #remove the one we just checked
        popfirst!(spins_to_check)
    end
    for i in cluster
        spins_angles[i] = mod((π +2*α - spins_angles[i]),2*π)
    end
    return spins_angles
end

In [49]:
function XY_cluster_update(config,energy,interactions,beta,energy_func) #(L::Int, neighbours, spins_angles, T) #
    # angle defining flipping axis
    N = length(config)
    alpha = rand(Uniform(0,2*pi))
    cluster = []
    spins_to_check = []
    initial_spin = rand(1:N)
    append!(cluster, initial_spin)
    append!(spins_to_check, initial_spin)
    while !isempty(spins_to_check)
        # check all neighbours of spins_to_check
        site = spins_to_check[1]
        theta = config[site]
        neighborsx = interactions[!,"$site,x"][1,1]
        neighborsy = interactions[!,"$site,y"][1,1]
        neighbors = append!(copy(neighborsx),neighborsy)
        neighbors_not_in_cluster = setdiff(neighbors,cluster)
        for i in neighbors_not_in_cluster
            theta_neighbor = config[i]
            prob = 1-exp(min(0,-(2*beta)*cos(theta-alpha)*cos(theta_neighbor-alpha)))
            if prob > rand()
                append!(cluster, i)
                append!(spins_to_check, i)
            end
        end
        #remove the one we just checked
        popfirst!(spins_to_check)
    end
    
    for i in cluster
        config[i] = mod((pi +2*alpha - config[i]),2*pi)
    end
    
    new_energy = energy_func(config,interactions)
    deltaE = new_energy-energy
    delta_accept_count = 0
    
    return config, deltaE, delta_accept_count
end

XY_cluster_update (generic function with 1 method)

In [11]:
function spin_stiffness(beta,config,interactions,rows,cols)
    
    N = length(config)
    L = Int(sqrt(N)) # obv for a square lattice only
    
    avg_ps_over_rows = 0
    for row in 1:L
        sites_in_row = rows[!,"row$row"]
        cos_contributions_x = 0
        sin_contributions_x = 0
        for site in sites_in_row
            neighbors_x = interactions[!,"$site,x"][1,1]
            angle_diff_x = config[site].-config[neighbors_x]
            cos_contributions_x += sum(0.5*cos.(angle_diff_x))
            sin_contributions_x += sum(0.5*sin.(angle_diff_x))
        end
        avg_ps_over_rows += ((1/N) * cos_contributions_x) - ((beta/N) * sin_contributions_x^2)
    end
    
    avg_ps_over_cols = 0
    for col in 1:L
        sites_in_cols = cols[!,"col$col"]
        cos_contributions_y = 0
        sin_contributions_y = 0
        for site in sites_in_cols
            neighbors_y = interactions[!,"$site,y"][1,1]
            angle_diff_y = config[site].-config[neighbors_y]
            cos_contributions_y += sum(0.5*cos.(angle_diff_y))
            sin_contributions_y += sum(0.5*sin.(angle_diff_y))
        end
        avg_ps_over_cols += ((1/N) * cos_contributions_y) - ((beta/N) * sin_contributions_y^2)
    end
    
    spin_stiffness = (avg_ps_over_rows+avg_ps_over_cols)/2
            
    return spin_stiffness, avg_ps_over_rows, avg_ps_over_cols
end

spin_stiffness (generic function with 1 method)

In [73]:
function MonteCarlo_XY(beta,Lx,Ly,warmup_steps,steps,build_lattice_function,rows_and_cols_function,update_function,energy_function,spin_stiffness_function)
    
    # system information
    N = Lx*Ly
    interactions = build_lattice_function(Lx,Ly)
    rows,cols = rows_and_cols_function(Lx,Ly)
    initial_config = rand(Float64,N)*2*pi
    
    # initialize dataframes
    maindf = DataFrame(:avenergy => Float64[],:avenergy2 => Float64[],:pss => Float64[],:avpss => Float64[],:accept_rate => Float64[])
    configs = DataFrame()
    
    # warmup steps - track Es and Ms but not in averages
    config = initial_config
    configs[!,:initial] = config
    energy = energy_function(config,interactions)
    for step in (1:warmup_steps)
        config, deltaE, _ = update_function(config,energy,interactions,beta,energy_function)
        energy = energy+deltaE
    end
    configs[!,:after_warmup] = config
    
    # begin MC steps and observable tracking
    sumE = 0
    sumE2 = 0
    sumps = 0
    accept_count = 0
    for step in (1:steps)
        new_config,deltaE,delta_ac = update_function(config,energy,interactions,beta,energy_function)
        config = new_config
        new_energy = energy+deltaE
        energy = new_energy
        accept_count += delta_ac
        
        # calculations
        sumE = sumE+energy
        sumE2 = sumE2+energy^2
        ps,_,_ = spin_stiffness_function(beta,config,interactions,rows,cols)
        sumps += ps
        push!(maindf, (sumE/(step*N),sumE2/step,ps,sumps/step,accept_count/(step)))
    end
    configs[!,:final] = config
       
    return maindf, configs
end

MonteCarlo_XY (generic function with 1 method)

In [50]:
MonteCarlo_XY(100,4,4,500,1000,build_lattice_df,rows_and_columns,XY_cluster_update,Calculate_XY_Energy,spin_stiffness)


Warmup steps over!


(1000×5 DataFrame
  Row │ avenergy  avenergy2  pss       avpss     accept_rate 
      │ Float64   Float64    Float64   Float64   Float64     
──────┼──────────────────────────────────────────────────────
    1 │ -1.99571    1019.61  0.997853  0.997853          0.0
    2 │ -1.99571    1019.61  0.997853  0.997853          0.0
    3 │ -1.99571    1019.61  0.997853  0.997853          0.0
    4 │ -1.99571    1019.61  0.997853  0.997853          0.0
    5 │ -1.99571    1019.61  0.997853  0.997853          0.0
    6 │ -1.99571    1019.61  0.997853  0.997853          0.0
    7 │ -1.99549    1019.38  0.997091  0.997744          0.0
    8 │ -1.99532    1019.22  0.997091  0.997662          0.0
    9 │ -1.9952     1019.09  0.997091  0.997599          0.0
   10 │ -1.9951     1018.98  0.997091  0.997548          0.0
   11 │ -1.99501    1018.9   0.997091  0.997506          0.0
  ⋮   │    ⋮          ⋮         ⋮         ⋮           ⋮
  991 │ -1.9956     1019.5   0.997401  0.997799          0.0
  992 │ 

# Part 2 Runs

In [76]:
Ts = collect(0.2:0.01:2.0)
Ls = [8,10,12,14,16]
N_chains = 10
n_steps = 1000
warmup = 500

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        T = round(T,digits=3)
        println("T = ",T," and beta = ",beta)
        for chain in 1:N_chains
            println("chain #",chain)
            @time begin
            avgs,configs = MonteCarlo_XY(beta,L,L,warmup,n_steps,build_lattice_df,rows_and_columns,XY_cluster_update,Calculate_XY_Energy,spin_stiffness);
            end
            CSV.write("./data/q2_cluster/L_$(L)/avgvals_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
            CSV.write("./data/q2_cluster/L_$(L)/configs_T_$(T)_chain$(chain).csv",  configs, writeheader=true)
        end
    end
end

L = 8
T = 0.2 and beta = 5.0
chain #1
  1.366882 seconds (12.29 M allocations: 410.586 MiB, 10.74% gc time)
chain #2
  1.046060 seconds (12.11 M allocations: 405.808 MiB, 4.37% gc time)
chain #3
  1.024508 seconds (11.96 M allocations: 402.088 MiB, 3.69% gc time)
chain #4
  1.033224 seconds (12.13 M allocations: 406.383 MiB, 3.68% gc time)
chain #5
  1.035823 seconds (12.12 M allocations: 406.167 MiB, 3.61% gc time)
chain #6
  1.039213 seconds (12.10 M allocations: 405.712 MiB, 3.62% gc time)
chain #7
  1.036054 seconds (12.07 M allocations: 404.772 MiB, 3.68% gc time)
chain #8
  1.065843 seconds (12.11 M allocations: 405.844 MiB, 4.01% gc time)
chain #9
  1.045440 seconds (12.05 M allocations: 404.302 MiB, 3.79% gc time)
chain #10
  1.070567 seconds (12.24 M allocations: 409.464 MiB, 3.77% gc time)
T = 0.21 and beta = 4.761904761904762
chain #1
  1.051504 seconds (11.95 M allocations: 401.791 MiB, 3.70% gc time)
chain #2
  1.043634 seconds (12.04 M allocations: 403.978 MiB, 3.63% gc t

  1.175435 seconds (11.71 M allocations: 395.377 MiB, 3.60% gc time)
chain #2
  1.171567 seconds (11.81 M allocations: 398.148 MiB, 2.89% gc time)
chain #3
  1.194094 seconds (11.72 M allocations: 395.820 MiB, 3.78% gc time)
chain #4
  1.173863 seconds (11.80 M allocations: 397.695 MiB, 3.54% gc time)
chain #5
  1.165373 seconds (11.78 M allocations: 397.389 MiB, 2.88% gc time)
chain #6
  1.205198 seconds (11.93 M allocations: 401.292 MiB, 3.62% gc time)
chain #7
  1.202515 seconds (11.85 M allocations: 398.996 MiB, 3.57% gc time)
chain #8
  1.211187 seconds (11.76 M allocations: 396.885 MiB, 2.97% gc time)
chain #9
  1.187316 seconds (11.82 M allocations: 398.227 MiB, 3.56% gc time)
chain #10
  1.163935 seconds (11.78 M allocations: 397.420 MiB, 2.99% gc time)
T = 0.31 and beta = 3.2258064516129035
chain #1
  1.177176 seconds (11.70 M allocations: 395.057 MiB, 3.02% gc time)
chain #2
  1.183520 seconds (11.68 M allocations: 394.757 MiB, 3.89% gc time)
chain #3
  1.164646 seconds (11.7

  1.178780 seconds (11.24 M allocations: 383.197 MiB, 2.61% gc time)
chain #2
  1.193256 seconds (11.09 M allocations: 379.120 MiB, 3.56% gc time)
chain #3
  1.205226 seconds (11.22 M allocations: 382.706 MiB, 3.37% gc time)
chain #4
  1.210592 seconds (11.13 M allocations: 380.182 MiB, 3.29% gc time)
chain #5
  1.166306 seconds (11.17 M allocations: 381.199 MiB, 3.23% gc time)
chain #6
  1.192270 seconds (11.15 M allocations: 380.836 MiB, 3.38% gc time)
chain #7
  1.179488 seconds (11.18 M allocations: 381.583 MiB, 2.65% gc time)
chain #8
  1.176158 seconds (11.24 M allocations: 383.156 MiB, 2.51% gc time)
chain #9
  1.154794 seconds (11.07 M allocations: 378.716 MiB, 3.36% gc time)
chain #10
  1.161458 seconds (11.07 M allocations: 378.735 MiB, 3.37% gc time)
T = 0.51 and beta = 1.9607843137254901
chain #1
  1.200912 seconds (11.00 M allocations: 376.787 MiB, 3.96% gc time)
chain #2
  1.168146 seconds (11.22 M allocations: 382.396 MiB, 3.24% gc time)
chain #3
  1.154464 seconds (11.1

  1.148100 seconds (10.79 M allocations: 371.169 MiB, 2.65% gc time)
chain #2
  1.157423 seconds (10.83 M allocations: 372.147 MiB, 3.62% gc time)
chain #3
  1.145321 seconds (10.91 M allocations: 374.419 MiB, 2.85% gc time)
chain #4
  1.183760 seconds (10.89 M allocations: 373.995 MiB, 4.49% gc time)
chain #5
  1.175574 seconds (10.79 M allocations: 371.253 MiB, 2.57% gc time)
chain #6
  1.156681 seconds (10.84 M allocations: 372.413 MiB, 3.72% gc time)
chain #7
  1.151324 seconds (10.95 M allocations: 375.287 MiB, 2.62% gc time)
chain #8
  1.151734 seconds (10.86 M allocations: 373.091 MiB, 3.59% gc time)
chain #9
  1.143137 seconds (10.81 M allocations: 371.667 MiB, 2.67% gc time)
chain #10
  1.142063 seconds (10.78 M allocations: 371.124 MiB, 2.86% gc time)
T = 0.61 and beta = 1.639344262295082
chain #1
  1.108398 seconds (10.72 M allocations: 369.453 MiB, 2.68% gc time)
chain #2
  1.120287 seconds (10.88 M allocations: 373.599 MiB, 3.68% gc time)
chain #3
  1.097319 seconds (10.82

  1.057523 seconds (10.21 M allocations: 355.886 MiB, 3.76% gc time)
chain #2
  1.041759 seconds (10.10 M allocations: 353.207 MiB, 3.76% gc time)
chain #3
  1.047758 seconds (10.32 M allocations: 359.039 MiB, 3.64% gc time)
chain #4
  1.024998 seconds (10.16 M allocations: 354.683 MiB, 3.21% gc time)
chain #5
  1.036751 seconds (10.11 M allocations: 353.649 MiB, 3.13% gc time)
chain #6
  1.067803 seconds (10.29 M allocations: 358.068 MiB, 3.71% gc time)
chain #7
  1.033315 seconds (10.18 M allocations: 355.350 MiB, 3.79% gc time)
chain #8
  1.051171 seconds (10.26 M allocations: 357.315 MiB, 3.76% gc time)
chain #9
  1.019785 seconds (10.07 M allocations: 352.498 MiB, 3.12% gc time)
chain #10
  1.051024 seconds (10.33 M allocations: 359.003 MiB, 3.78% gc time)
T = 0.81 and beta = 1.2345679012345678
chain #1
  1.032389 seconds (10.07 M allocations: 352.470 MiB, 3.81% gc time)
chain #2
  1.023074 seconds (10.13 M allocations: 353.991 MiB, 3.13% gc time)
chain #3
  1.038471 seconds (10.2

...

  3.305698 seconds (38.16 M allocations: 1.152 GiB, 3.28% gc time)
chain #3
  3.232339 seconds (37.03 M allocations: 1.127 GiB, 3.41% gc time)
chain #4
  3.218114 seconds (37.45 M allocations: 1.136 GiB, 3.28% gc time)
chain #5
  3.204524 seconds (37.06 M allocations: 1.129 GiB, 3.40% gc time)
chain #6
  3.303234 seconds (37.69 M allocations: 1.142 GiB, 3.28% gc time)
chain #7
  3.314337 seconds (37.54 M allocations: 1.138 GiB, 3.22% gc time)
chain #8
  3.305392 seconds (37.40 M allocations: 1.136 GiB, 3.25% gc time)
chain #9
  3.307174 seconds (37.74 M allocations: 1.143 GiB, 3.21% gc time)
chain #10
  3.326991 seconds (37.19 M allocations: 1.131 GiB, 3.23% gc time)
T = 0.82 and beta = 1.2195121951219512
chain #1
  3.294681 seconds (37.18 M allocations: 1.131 GiB, 3.23% gc time)
chain #2
  3.309769 seconds (37.16 M allocations: 1.131 GiB, 3.39% gc time)
chain #3
  3.257946 seconds (36.75 M allocations: 1.122 GiB, 3.11% gc time)
chain #4
  3.358341 seconds (38.10 M allocations: 1.150 G

  3.067361 seconds (30.96 M allocations: 1.006 GiB, 3.01% gc time)
chain #7
  3.063467 seconds (31.06 M allocations: 1.008 GiB, 2.96% gc time)
chain #8
  3.055772 seconds (31.14 M allocations: 1.009 GiB, 2.68% gc time)
chain #9
  3.029372 seconds (30.18 M allocations: 1013.278 MiB, 2.89% gc time)
chain #10
  3.089448 seconds (31.23 M allocations: 1.010 GiB, 2.94% gc time)
T = 1.02 and beta = 0.9803921568627451
chain #1
  3.053164 seconds (30.15 M allocations: 1012.028 MiB, 2.81% gc time)
chain #2
  3.065767 seconds (30.09 M allocations: 1011.568 MiB, 3.02% gc time)
chain #3
  3.009544 seconds (30.13 M allocations: 1012.774 MiB, 2.79% gc time)
chain #4
  3.024572 seconds (30.08 M allocations: 1011.300 MiB, 3.07% gc time)
chain #5
  3.068954 seconds (30.66 M allocations: 1023.044 MiB, 2.74% gc time)
chain #6
  3.027150 seconds (29.97 M allocations: 1008.064 MiB, 2.85% gc time)
chain #7
  3.007123 seconds (30.87 M allocations: 1.004 GiB, 2.90% gc time)
chain #8
  2.981649 seconds (30.87 M

  2.637157 seconds (25.65 M allocations: 917.265 MiB, 3.45% gc time)
chain #7
  2.657299 seconds (26.14 M allocations: 927.965 MiB, 3.19% gc time)
chain #8
  2.723912 seconds (25.68 M allocations: 917.756 MiB, 3.17% gc time)
chain #9
  2.732275 seconds (26.03 M allocations: 925.659 MiB, 3.48% gc time)
chain #10
  2.715632 seconds (25.97 M allocations: 924.361 MiB, 3.08% gc time)
T = 1.12 and beta = 0.8928571428571428
chain #1
  2.722659 seconds (25.48 M allocations: 914.215 MiB, 3.16% gc time)
chain #2
  2.669054 seconds (25.15 M allocations: 906.864 MiB, 3.16% gc time)
chain #3
  2.719401 seconds (25.56 M allocations: 916.181 MiB, 3.14% gc time)
chain #4
  2.750045 seconds (26.25 M allocations: 930.329 MiB, 3.66% gc time)
chain #5
  2.704527 seconds (25.57 M allocations: 915.031 MiB, 3.15% gc time)
chain #6
  2.741469 seconds (26.03 M allocations: 926.203 MiB, 3.18% gc time)
chain #7
  2.714891 seconds (25.66 M allocations: 917.631 MiB, 3.14% gc time)
chain #8
  2.758467 seconds (26.3

Excessive output truncated after 524288 bytes.

  2.525119 seconds (22.72 M allocations: 853.480 MiB, 3.41% gc time)


In [77]:
Ts = collect(0.2:0.01:2.0)
Ls = [20,24]
N_chains = 10
n_steps = 1000
warmup = 500

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        T = round(T,digits=3)
        println("T = ",T," and beta = ",beta)
        for chain in 1:N_chains
            println("chain #",chain)
            @time begin
            avgs,configs = MonteCarlo_XY(beta,L,L,warmup,n_steps,build_lattice_df,rows_and_columns,XY_cluster_update,Calculate_XY_Energy,spin_stiffness);
            end
            CSV.write("./data/q2_cluster/L_$(L)/avgvals_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
#             CSV.write("./data/q2_cluster/L_$(L)/configs_T_$(T)_chain$(chain).csv",  configs, writeheader=true)
        end
    end
end

L = 20
T = 0.2 and beta = 5.0
chain #1
  9.647859 seconds (150.54 M allocations: 3.557 GiB, 4.78% gc time)
chain #2
  9.536391 seconds (150.45 M allocations: 3.554 GiB, 4.81% gc time)
chain #3
  9.447629 seconds (150.91 M allocations: 3.564 GiB, 4.75% gc time)
chain #4
  9.528777 seconds (153.14 M allocations: 3.603 GiB, 4.70% gc time)
chain #5
  9.556699 seconds (153.56 M allocations: 3.611 GiB, 4.77% gc time)
chain #6
  9.729251 seconds (154.40 M allocations: 3.626 GiB, 4.71% gc time)
chain #7
 10.556181 seconds (151.94 M allocations: 3.583 GiB, 4.68% gc time)
chain #8
 11.170065 seconds (149.60 M allocations: 3.540 GiB, 4.45% gc time)
chain #9
 11.439891 seconds (152.56 M allocations: 3.593 GiB, 4.51% gc time)
chain #10
 11.203096 seconds (148.86 M allocations: 3.526 GiB, 4.38% gc time)
T = 0.21 and beta = 4.761904761904762
chain #1
 11.329505 seconds (151.12 M allocations: 3.568 GiB, 4.45% gc time)
chain #2
 11.314035 seconds (150.39 M allocations: 3.554 GiB, 4.38% gc time)
chain #

  9.088657 seconds (140.26 M allocations: 3.375 GiB, 4.21% gc time)
chain #3
  9.097017 seconds (140.82 M allocations: 3.386 GiB, 4.19% gc time)
chain #4
  8.909803 seconds (140.35 M allocations: 3.376 GiB, 4.24% gc time)
chain #5
  8.833902 seconds (140.54 M allocations: 3.378 GiB, 4.16% gc time)
chain #6
  8.919747 seconds (142.28 M allocations: 3.412 GiB, 4.25% gc time)
chain #7
  8.828557 seconds (139.76 M allocations: 3.366 GiB, 4.30% gc time)
chain #8
  8.829520 seconds (140.17 M allocations: 3.373 GiB, 4.17% gc time)
chain #9
  8.844146 seconds (140.48 M allocations: 3.379 GiB, 4.30% gc time)
chain #10
  8.832291 seconds (140.24 M allocations: 3.376 GiB, 4.16% gc time)
T = 0.31 and beta = 3.2258064516129035
chain #1
  8.840510 seconds (140.17 M allocations: 3.373 GiB, 4.30% gc time)
chain #2
  8.777332 seconds (139.37 M allocations: 3.360 GiB, 4.20% gc time)
chain #3
  8.727364 seconds (137.76 M allocations: 3.329 GiB, 4.24% gc time)
chain #4
  8.818596 seconds (139.66 M allocat

  8.051936 seconds (123.62 M allocations: 3.081 GiB, 3.44% gc time)
chain #6
  7.667951 seconds (121.27 M allocations: 3.037 GiB, 3.62% gc time)
chain #7
  7.690533 seconds (121.95 M allocations: 3.052 GiB, 3.47% gc time)
chain #8
  7.885071 seconds (120.49 M allocations: 3.024 GiB, 3.45% gc time)
chain #9
  7.947712 seconds (121.57 M allocations: 3.044 GiB, 3.44% gc time)
chain #10
  7.413439 seconds (121.46 M allocations: 3.043 GiB, 3.57% gc time)
T = 0.51 and beta = 1.9607843137254901
chain #1
  7.943379 seconds (121.55 M allocations: 3.044 GiB, 3.46% gc time)
chain #2
  8.103851 seconds (124.81 M allocations: 3.101 GiB, 3.45% gc time)
chain #3
  7.772195 seconds (123.36 M allocations: 3.076 GiB, 3.46% gc time)
chain #4
  7.610740 seconds (120.33 M allocations: 3.021 GiB, 3.54% gc time)
chain #5
  7.970275 seconds (122.33 M allocations: 3.056 GiB, 3.44% gc time)
chain #6
  7.982974 seconds (122.13 M allocations: 3.054 GiB, 3.48% gc time)
chain #7
  7.355049 seconds (120.50 M allocat

  7.730946 seconds (115.13 M allocations: 2.932 GiB, 3.56% gc time)
chain #8
  7.127002 seconds (115.50 M allocations: 2.937 GiB, 3.74% gc time)
chain #9
  7.970017 seconds (118.18 M allocations: 2.987 GiB, 3.61% gc time)
chain #10
  7.752953 seconds (116.06 M allocations: 2.947 GiB, 3.61% gc time)
T = 0.61 and beta = 1.639344262295082
chain #1
  7.401190 seconds (113.06 M allocations: 2.894 GiB, 3.65% gc time)
chain #2
  7.315140 seconds (114.30 M allocations: 2.915 GiB, 3.64% gc time)
chain #3
  7.527121 seconds (111.18 M allocations: 2.860 GiB, 3.64% gc time)
chain #4
  7.478738 seconds (116.28 M allocations: 2.952 GiB, 3.56% gc time)
chain #5
  7.454719 seconds (115.48 M allocations: 2.937 GiB, 3.74% gc time)
chain #6
  7.547891 seconds (111.57 M allocations: 2.865 GiB, 3.51% gc time)
chain #7
  7.351295 seconds (111.55 M allocations: 2.867 GiB, 3.71% gc time)
chain #8
  7.328348 seconds (114.71 M allocations: 2.923 GiB, 3.75% gc time)
chain #9
  7.728761 seconds (113.67 M allocati

  7.331046 seconds (105.98 M allocations: 2.767 GiB, 3.75% gc time)
chain #10
  7.049888 seconds (105.62 M allocations: 2.761 GiB, 3.84% gc time)
T = 0.71 and beta = 1.4084507042253522
chain #1
  6.990735 seconds (105.76 M allocations: 2.765 GiB, 3.87% gc time)
chain #2
  7.244798 seconds (104.62 M allocations: 2.741 GiB, 3.83% gc time)
chain #3
  7.196877 seconds (106.71 M allocations: 2.781 GiB, 3.91% gc time)
chain #4
  6.913762 seconds (105.37 M allocations: 2.757 GiB, 3.94% gc time)
chain #5
  7.228491 seconds (104.01 M allocations: 2.733 GiB, 3.69% gc time)
chain #6
  7.240633 seconds (103.74 M allocations: 2.728 GiB, 3.85% gc time)
chain #7
  6.547578 seconds (102.11 M allocations: 2.698 GiB, 3.91% gc time)
chain #8
  7.200822 seconds (103.27 M allocations: 2.721 GiB, 3.86% gc time)
chain #9
  7.279089 seconds (105.21 M allocations: 2.754 GiB, 3.82% gc time)
chain #10
  6.605692 seconds (102.94 M allocations: 2.713 GiB, 3.95% gc time)
T = 0.72 and beta = 1.3888888888888888
chain

...

  6.092953 seconds (57.26 M allocations: 2.219 GiB, 3.02% gc time)
chain #8
  5.292221 seconds (56.74 M allocations: 2.209 GiB, 3.29% gc time)
chain #9
  5.313060 seconds (56.91 M allocations: 2.212 GiB, 3.31% gc time)
chain #10
  6.297198 seconds (58.02 M allocations: 2.235 GiB, 2.98% gc time)
T = 1.25 and beta = 0.8
chain #1
  5.389921 seconds (56.69 M allocations: 2.208 GiB, 3.17% gc time)
chain #2
  5.331010 seconds (56.86 M allocations: 2.212 GiB, 3.33% gc time)
chain #3
  5.892952 seconds (56.67 M allocations: 2.208 GiB, 3.04% gc time)
chain #4
  5.684658 seconds (57.11 M allocations: 2.216 GiB, 3.23% gc time)
chain #5
  5.298460 seconds (56.04 M allocations: 2.195 GiB, 3.34% gc time)
chain #6
  5.358567 seconds (57.38 M allocations: 2.222 GiB, 3.31% gc time)
chain #7
  6.168747 seconds (56.57 M allocations: 2.205 GiB, 2.99% gc time)
chain #8
  5.326782 seconds (56.49 M allocations: 2.204 GiB, 3.31% gc time)
chain #9
  5.339203 seconds (56.95 M allocations: 2.213 GiB, 3.30% gc ti

  6.003739 seconds (55.01 M allocations: 2.172 GiB, 3.13% gc time)
T = 1.35 and beta = 0.7407407407407407
chain #1
  5.459782 seconds (54.76 M allocations: 2.166 GiB, 3.36% gc time)
chain #2
  5.287901 seconds (54.93 M allocations: 2.170 GiB, 3.45% gc time)
chain #3
  5.561343 seconds (55.01 M allocations: 2.172 GiB, 3.35% gc time)
chain #4
  5.931589 seconds (55.14 M allocations: 2.174 GiB, 3.04% gc time)
chain #5
  5.280970 seconds (54.98 M allocations: 2.171 GiB, 3.50% gc time)
chain #6
  5.302992 seconds (55.17 M allocations: 2.176 GiB, 3.43% gc time)
chain #7
  5.962933 seconds (54.95 M allocations: 2.171 GiB, 3.21% gc time)
chain #8
  5.584800 seconds (55.06 M allocations: 2.173 GiB, 3.25% gc time)
chain #9
  5.327128 seconds (54.96 M allocations: 2.170 GiB, 3.48% gc time)
chain #10
  5.302490 seconds (55.20 M allocations: 2.176 GiB, 3.32% gc time)
T = 1.36 and beta = 0.7352941176470588
chain #1
  6.131833 seconds (54.89 M allocations: 2.169 GiB, 3.20% gc time)
chain #2
  5.30059

  5.207208 seconds (54.12 M allocations: 2.150 GiB, 2.92% gc time)
chain #5
  5.301815 seconds (54.13 M allocations: 2.151 GiB, 3.04% gc time)
chain #6
  6.063162 seconds (54.15 M allocations: 2.151 GiB, 2.70% gc time)
chain #7
  5.214410 seconds (54.10 M allocations: 2.150 GiB, 3.14% gc time)
chain #8
  5.202404 seconds (54.13 M allocations: 2.151 GiB, 2.97% gc time)
chain #9
  5.920580 seconds (54.17 M allocations: 2.152 GiB, 2.75% gc time)
chain #10
  5.452706 seconds (54.19 M allocations: 2.152 GiB, 3.06% gc time)
T = 1.56 and beta = 0.641025641025641
chain #1
  5.164924 seconds (54.16 M allocations: 2.151 GiB, 3.00% gc time)
chain #2
  5.584420 seconds (54.12 M allocations: 2.150 GiB, 2.87% gc time)
chain #3
  5.726470 seconds (54.19 M allocations: 2.152 GiB, 2.83% gc time)
chain #4
  5.194236 seconds (54.18 M allocations: 2.152 GiB, 3.00% gc time)
chain #5
  5.195051 seconds (54.13 M allocations: 2.151 GiB, 3.04% gc time)
chain #6
  6.060663 seconds (54.17 M allocations: 2.152 Gi

  5.801020 seconds (53.86 M allocations: 2.144 GiB, 2.86% gc time)
chain #9
  5.514614 seconds (53.89 M allocations: 2.144 GiB, 2.96% gc time)
chain #10
  5.142848 seconds (53.87 M allocations: 2.144 GiB, 3.10% gc time)
T = 1.76 and beta = 0.5681818181818182
chain #1
  5.536504 seconds (53.87 M allocations: 2.144 GiB, 2.98% gc time)
chain #2
  5.845429 seconds (53.87 M allocations: 2.144 GiB, 2.80% gc time)
chain #3
  5.206662 seconds (53.89 M allocations: 2.144 GiB, 3.09% gc time)
chain #4
  5.167690 seconds (53.90 M allocations: 2.145 GiB, 3.00% gc time)
chain #5
  6.064184 seconds (53.89 M allocations: 2.144 GiB, 2.80% gc time)
chain #6
  5.246347 seconds (53.85 M allocations: 2.143 GiB, 2.96% gc time)
chain #7
  5.201978 seconds (53.90 M allocations: 2.145 GiB, 2.99% gc time)
chain #8
  5.861023 seconds (53.93 M allocations: 2.145 GiB, 2.82% gc time)
chain #9
  5.499066 seconds (53.91 M allocations: 2.145 GiB, 2.80% gc time)
chain #10
  5.183658 seconds (53.93 M allocations: 2.145 

  5.501177 seconds (53.79 M allocations: 2.142 GiB, 3.31% gc time)
chain #4
  5.907358 seconds (53.80 M allocations: 2.142 GiB, 3.19% gc time)
chain #5
  5.226490 seconds (53.79 M allocations: 2.142 GiB, 3.46% gc time)
chain #6
  5.220667 seconds (53.80 M allocations: 2.142 GiB, 3.25% gc time)
chain #7
  6.329420 seconds (53.78 M allocations: 2.141 GiB, 7.24% gc time)
chain #8
  5.367657 seconds (53.76 M allocations: 2.141 GiB, 3.87% gc time)
chain #9
  5.361560 seconds (53.79 M allocations: 2.142 GiB, 3.89% gc time)
chain #10
  5.464743 seconds (53.80 M allocations: 2.142 GiB, 3.76% gc time)
T = 1.97 and beta = 0.5076142131979695
chain #1
  6.016416 seconds (53.78 M allocations: 2.141 GiB, 3.44% gc time)
chain #2
  5.298624 seconds (53.77 M allocations: 2.141 GiB, 3.78% gc time)
chain #3
  5.281016 seconds (53.79 M allocations: 2.142 GiB, 3.64% gc time)
chain #4
  5.675399 seconds (53.77 M allocations: 2.141 GiB, 3.42% gc time)
chain #5
  5.763267 seconds (53.76 M allocations: 2.141 G